In [1]:
import SimPEG as simpeg
import simpegEM as simpegem, simpegMT as simpegmt
from SimPEG.Utils import meshTensor
import numpy as np
import simpegMT.Tests.test_Problem3D_againstAnalytic as t3Dmt

In [2]:
sigmaHalf = 0.01
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.halfSpace(sigmaHalf),comp='zxyi',singleFreq=1)
if False:
    fields = problem.fields(problem.curModel.sigma)
    data = survey.dpred(problem.curModel.sigma)

In [3]:
def testDeriv_dA_dm(prb,cond):
    TOL = 1e-4
    FLR = 1e-20
    x0 = np.log(np.ones(prb.mesh.nC)*cond)
    prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
    if True:
        x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    survey = prb.survey
    src = survey.srcList[0]
    freq = src.freq
    v1 = np.random.randn(prb.mesh.nE,1)
    v2 = np.random.randn(prb.mesh.nE,1)
    v = np.hstack(( simpeg.mkvc(v1,2), simpeg.mkvc(v2,2)))
    u_0 = prb.fieldsPair(prb.mesh,prb.survey)
    u_0[src,'e_pxSolution'] = v1
    u_0[src,'e_pySolution'] = v2
    def fun(x):
        prb.curModel = x
        A = prb.getA(freq) #
#         return simpeg.mkvc(A*v1)+simpeg.mkvc(A*v2), lambda t: simpeg.mkvc(prb.getADeriv_m(freq, u_0[src], t))
        return A*v, lambda t: (prb.getADeriv_m(freq, u_0[src], t))
    return simpeg.Tests.checkDerivative(fun, x0, num=3, plotIt=False, eps=FLR)
testDeriv_dA_dm(problem,0.1)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.111e+11     3.481e+10      nan
 1   1.00e-02    3.112e+10     3.421e+08      2.008
 2   1.00e-03    3.116e+09     3.420e+06      2.000
========================= PASS! =========================
Once upon a time, a happy little test passed.



True

In [4]:
def testDeriv_dRHS_dm(prb,cond):
    TOL = 1e-4
    FLR = 1e-20
    x0 = np.log(np.ones(prb.mesh.nC)*cond)
    prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
    if True:
        x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    survey = prb.survey
    src = survey.srcList[0]
    
    u0 = prb.fields(x0)
    freq = src.freq
    A = prb.getA(freq) #
    A_I = prb.Solver(A, **prb.solverOpts)

    ftype = prb._fieldType + 'Solution'
    u0_src = u0[src, ftype]
    v = np.random.randn(prb.mesh.nE,1)
    
    def fun(x):
        prb.curModel = x
        return simpeg.mkvc(np.sum(prb.getRHS(freq))), lambda x: simpeg.mkvc(prb.getRHSDeriv_m(freq, x))
#         return simpeg.mkvc(prb.fields(x)[src,prb._fieldType + 'Solution']), lambda x: simpeg.mkvc(prb.getADeriv_m(freq, u0_src, x))
    return simpeg.Tests.checkDerivative(fun, x0, num=3, plotIt=False, eps=FLR)
testDeriv_dA_dm(problem,0.1)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.187e+11     3.398e+10      nan
 1   1.00e-02    3.176e+10     3.369e+08      2.004
 2   1.00e-03    3.178e+09     3.372e+06      2.000
========================= PASS! =========================
That was easy!



True

In [5]:
def testDeriv_S_e(prb,cond):
    # Initate things for the derivs Test
    TOL = 1e-4
    FLR = 1e-20
    
    src = prb.survey.srcList[0]
    rx = src.rxList[0]

    x0 = np.log(np.ones(prb.mesh.nC)*cond)
#     prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
    if True:
        x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    def fun(x):
        prb.curModel = x
        return src.S_e(prb), lambda t: src.S_eDeriv_m(prb,t)
    simpeg.Tests.checkDerivative(fun,x0,num=3,plotIt=False)
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(1),comp='All',singleFreq=1)
testDeriv_S_e(problem,0.1)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.873e+10     3.211e+09      nan
 1   1.00e-02    2.879e+09     3.204e+07      2.001
 2   1.00e-03    2.882e+08     3.207e+05      2.000
========================= PASS! =========================
You are awesome.



In [6]:
def testDeriv_epx(prb):
    # Initate things for the derivs Test
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    # f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0
    # Run a test
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
    #     f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
        return f._e_px(f[src,'e_pxSolution'],[src]).ravel(), lambda t: f._e_pxDeriv_u([src],t)[:len(u0)/2].ravel()
    simpeg.Tests.checkDerivative(fun,u0,num=3,plotIt=False)
    
def testDeriv_epy(prb):
    # Initate things for the derivs Test
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    # f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0
    # Run a test
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
        return f._e_py(f[src,'e_pySolution'],[src]).ravel(), lambda t: f._e_pyDeriv_u([src],t).ravel()
    simpeg.Tests.checkDerivative(fun,u0,num=3,plotIt=False)
    
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.1),comp='zxyr',singleFreq=.01)
testDeriv_epx(problem)
testDeriv_epy(problem)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.436e+01     1.792e-11      nan
 1   1.00e-02    1.436e+00     1.792e-11      0.000
 2   1.00e-03    1.436e-01     1.730e-11      0.015
========================= PASS! =========================
That was easy!

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.433e+01     1.792e-11      nan
 1   1.00e-02    1.433e+00     1.776e-11      0.004
 2   1.00e-03    1.433e-01     1.763e-11      0.003
========================= PASS! =========================
The test be workin!



In [7]:
def testDeriv_bpx(prb):
    # Initate things for the derivs Test
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    # f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0
    # Run a test
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
    #     f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
        return f._b_px(f[src,'e_pxSolution'],[src]).ravel(), lambda t: f._b_pxDeriv_u(src,t).ravel()
    simpeg.Tests.checkDerivative(fun,u0,num=3,plotIt=False)
    
def testDeriv_bpy(prb):
    # Initate things for the derivs Test
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    # f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0
    # Run a test
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
        return f._b_py(f[src,'e_pySolution'],[src]).ravel(), lambda t: f._b_pyDeriv_u(src,t).ravel()
    simpeg.Tests.checkDerivative(fun,u0,num=3,plotIt=False)
    
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.1),comp='zyxr',singleFreq=.01)
testDeriv_bpx(problem)
testDeriv_bpy(problem)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.995e-01     1.107e-13      nan
 1   1.00e-02    3.995e-02     1.105e-13      0.001
 2   1.00e-03    3.995e-03     1.110e-13      -0.002
========================= PASS! =========================
The test be workin!

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    4.075e-01     1.086e-13      nan
 1   1.00e-02    4.075e-02     1.049e-13      0.015
 2   1.00e-03    4.075e-03     1.025e-13      0.010
========================= PASS! =========================
Happy little convergence test!



In [8]:
# %debug

In [9]:
from scipy.constants import mu_0
def testDeriv_Hd(prb):
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    # Run a testdef testDeriv_ZijN(rx):
    def getHdcomp(rx,f,v=None):
        if rx.locs.ndim == 3:
            eFLocs = rx.locs[:,:,0]
            bFLocs = rx.locs[:,:,1]
        else:
            eFLocs = rx.locs
            bFLocs = rx.locs
        # Get the projection
        Pbx = prb.mesh.getInterpolationMat(bFLocs,'Fx')
        Pby = prb.mesh.getInterpolationMat(bFLocs,'Fy')
        # Get the fields at location
        # px: x-polaration and py: y-polaration.
        hx_px = Pbx*f[src,'b_px']/mu_0
        hy_px = Pby*f[src,'b_px']/mu_0
        hx_py = Pbx*f[src,'b_py']/mu_0
        hy_py = Pby*f[src,'b_py']/mu_0
        # Derivatives as lambda functions
        spPe = simpeg.Utils.spzeros(rx.nD,prb.mesh.nE)
        spPb = simpeg.Utils.spzeros(rx.nD,prb.mesh.nF)

        # NOTE: Think b_p?Deriv_u should return a 2*nF size matrix
        hx_px_u = lambda vec: Pbx*f._b_pxDeriv_u(src,vec)/mu_0
        hy_px_u = lambda vec: Pby*f._b_pxDeriv_u(src,vec)/mu_0
        hx_py_u = lambda vec: Pbx*f._b_pyDeriv_u(src,vec)/mu_0
        hy_py_u = lambda vec: Pby*f._b_pyDeriv_u(src,vec)/mu_0
        # Update the input vector
        sdiag = lambda t: simpeg.Utils.sdiag(simpeg.mkvc(t,2))
        # Define the components of the derivative
        if v is not None:
            return (sdiag(hy_py)*hx_px_u(v)) + (sdiag(hx_px)*hy_py_u(v)) - (sdiag(hx_py)*hy_px_u(v)) - (sdiag(hy_px)*hx_py_u(v))
        else:
            return (sdiag(hx_px)*hy_py) - (sdiag(hx_py)*hy_px)
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
        return getHdcomp(rx,f), lambda t: getHdcomp(rx,f0,t)
    simpeg.Tests.checkDerivative(fun,u0,num=6,plotIt=False)
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.01),comp='zxyr',singleFreq=.0001)
testDeriv_Hd(problem)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    5.819e+11     7.675e+10      nan
 1   1.00e-02    5.440e+10     8.852e+08      1.938
 2   1.00e-03    5.403e+09     4.494e+07      1.294
 3   1.00e-04    5.399e+08     4.428e+06      1.006
 4   1.00e-05    5.399e+07     4.428e+05      1.000
 5   1.00e-06    5.399e+06     4.428e+04      1.000
*********************************************************
<<<<<<<<<<<<<<<<<<<<<<<<< FAIL! >>>>>>>>>>>>>>>>>>>>>>>>>
*********************************************************
You break it, you fix it.



In [10]:
def testDeriv_ZijN(prb):
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(prb.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(prb.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(prb.mesh,prb.survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    def getZijNcomp(rx,f,v=None):
        if rx.locs.ndim == 3:
            eFLocs = rx.locs[:,:,0]
            bFLocs = rx.locs[:,:,1]
        else:
            eFLocs = rx.locs
            bFLocs = rx.locs
        # Get the projection
        Pex = prb.mesh.getInterpolationMat(eFLocs,'Ex')
        Pey = prb.mesh.getInterpolationMat(eFLocs,'Ey')
        Pbx = prb.mesh.getInterpolationMat(bFLocs,'Fx')
        Pby = prb.mesh.getInterpolationMat(bFLocs,'Fy')
        # Get the fields at location
        # px: x-polaration and py: y-polaration.
        ex_px = Pex*f[src,'e_px']
        ey_px = Pey*f[src,'e_px']
        ex_py = Pex*f[src,'e_py']
        ey_py = Pey*f[src,'e_py']
        hx_px = Pbx*f[src,'b_px']/mu_0
        hy_px = Pby*f[src,'b_px']/mu_0
        hx_py = Pbx*f[src,'b_py']/mu_0
        hy_py = Pby*f[src,'b_py']/mu_0
        # Derivatives as lambda functions
        # The size of the deratives should be nD,nU
        ex_px_u = lambda vec: Pex*f._e_pxDeriv_u(src,vec)
        ey_px_u = lambda vec: Pey*f._e_pxDeriv_u(src,vec)
        ex_py_u = lambda vec: Pex*f._e_pyDeriv_u(src,vec)
        ey_py_u = lambda vec: Pey*f._e_pyDeriv_u(src,vec)
        # NOTE: Think b_p?Deriv_u should return a 2*nF size matrix
        hx_px_u = lambda vec: Pbx*f._b_pxDeriv_u(src,vec)/mu_0
        hy_px_u = lambda vec: Pby*f._b_pxDeriv_u(src,vec)/mu_0
        hx_py_u = lambda vec: Pbx*f._b_pyDeriv_u(src,vec)/mu_0
        hy_py_u = lambda vec: Pby*f._b_pyDeriv_u(src,vec)/mu_0

        # Update the input vector
        # Define the components of the derivative# Calculate components
        # Update the input vector
        sdiag = lambda t: simpeg.Utils.sdiag(simpeg.mkvc(t,2))
        # Define the components of the derivative
        if 'zxx' in rx.rxType:
            if v is not None:
                return  sdiag(hy_py)*ex_px_u(v) + sdiag(ex_px)*hy_py_u(v) - sdiag(ex_py)*hy_px_u(v) - sdiag(hy_px)*ex_py_u(v)
            return ( sdiag(ex_px)*hy_py - sdiag(ex_py)*hy_px)
        elif 'zxy' in rx.rxType:
            if v is not None:
                return -sdiag(hx_py)*ex_px_u(v) - sdiag(ex_px)*hx_py_u(v) + sdiag(ex_py)*hx_px_u(v) + sdiag(hx_px)*ex_py_u(v)
            return (-sdiag(ex_px)*hx_py + sdiag(ex_py)*hx_px)
        elif 'zyx' in rx.rxType:
            if v is not None:
                return  ey_px_u(v)*hy_py + ey_px*hy_py_u(v) - ey_py*hy_px_u(v) - ey_py_u(v)*hy_px
            return ( ey_px*hy_py - ey_py*hy_px)
        elif 'zyy' in rx.rxType:
            if v is not None:
                return -ey_px_u(v)*hx_py - ey_px*hx_py_u(v) + ey_py*hx_px_u(v) + ey_py_u(v)*hx_px
            return (-ey_px*hx_py + ey_py*hx_px)
        
    def fun(u):
        f = prb.fieldsPair(survey.mesh,prb.survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
        return getZijNcomp(rx,f), lambda t: getZijNcomp(rx,f0,t)
    simpeg.Tests.checkDerivative(fun,u0,num=6,plotIt=False)
            
surveyxx, problemxx = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.01),comp='zxxr',singleFreq=.001)
testDeriv_ZijN(problemxx)
surveyxy, problemxy = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.01),comp='zxyr',singleFreq=.0001)
testDeriv_ZijN(problemxy)
surveyyx, problemyx = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.01),comp='zyxr',singleFreq=.0001)
testDeriv_ZijN(problemyx)
surveyyy, problemyy = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.01),comp='zyyr',singleFreq=.0001)
testDeriv_ZijN(problemyy)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.733e+05     7.391e+02      nan
 1   1.00e-02    1.734e+04     6.892e+01      1.030
 2   1.00e-03    1.734e+03     6.896e+00      1.000
 3   1.00e-04    1.734e+02     6.896e-01      1.000
 4   1.00e-05    1.734e+01     6.897e-02      1.000
 5   1.00e-06    1.734e+00     6.897e-03      1.000
*********************************************************
<<<<<<<<<<<<<<<<<<<<<<<<< FAIL! >>>>>>>>>>>>>>>>>>>>>>>>>
*********************************************************
You break it, you fix it.

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.075e+05     3.072e+03      nan
 1   1.00e-02    3.057e+04     1.202e+02      1.407
 2   1.00e-03    3.055e+03     1.169e+01      1.012
 

In [11]:
# %debug

In [12]:
# dA_dm = problem.getADeriv_m(src.freq,u_src,w)

In [13]:
# dA_dm.shape

In [14]:
# a, b, sig, d, e = t3Dmt.halfSpace(.01)

In [40]:
def testDeriv_ProjFields(prb):
    # Initate things for the derivs Test
    src = prb.survey.srcList[0]
    rx = src.rxList[0]
#     rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
#     u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
#     u0 = np.r_[u0x, u0y]
#     f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
#     f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
#     f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y

#     prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
#     if True:
#         x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    survey = prb.survey
    src = survey.srcList[0]
    
    f0 = prb.fields(prb.curModel)
    u0 = np.r_[f0[src,'e_pxSolution'],f0[src,'e_pySolution']]
#     def fun(x):
#         prb.curModel = x
    # f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0
    # Run a test
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
    #     f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
        return rx.projectFields(src,survey.mesh,f), lambda t: rx.projectFieldsDeriv(src,survey.mesh,f0,t)
    simpeg.Tests.checkDerivative(fun,u0,num=8,plotIt=False)
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.halfSpace(1),comp='zxyr',singleFreq=.1)
testDeriv_ProjFields(problem)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    5.050e-06     1.906e-09      nan
 1   1.00e-02    5.049e-07     1.949e-11      1.990
 2   1.00e-03    5.048e-08     1.969e-13      1.996
 3   1.00e-04    5.048e-09     1.377e-15      2.155
 4   1.00e-05    5.048e-10     1.073e-15      0.108
 5   1.00e-06    5.049e-11     4.179e-15      -0.590
 6   1.00e-07    5.059e-12     1.079e-14      -0.412
 7   1.00e-08    5.092e-13     4.411e-15      0.388
========================= PASS! =========================
Yay passed!



In [36]:
# %debug

In [17]:
src = survey.srcList[0]
rx = src.rxList[0]

In [18]:
rx.locs = np.array([[0, 0, 0],[1,1,1]])

In [30]:
problem.curModel.sigma

array([ 19.30432593,   8.59301637,   7.75725432, ...,   2.18228759,
        16.72231401,   7.67029076])

In [19]:
u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j    
src = survey.srcList[0]
rx = src.rxList[0]
f0 = problem.fieldsPair(survey.mesh,survey)
u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
f0[src,'e_pxSolution'] = u0x
f0[src,'e_pySolution'] = u0y

In [20]:
f0._e_px(f0[src,'e_pxSolution'],[src]).shape

(20536, 1)

In [21]:
f0._e_pxDeriv_u([src],u0)[len(u0)/2::].shape

(0, 1)

In [22]:
simpeg.Utils.spzeros

In [23]:
rx.projectFields(src,survey.mesh,f0)

array([[ 0.00031425],
       [ 0.00030875]])

In [24]:
%debug

ERROR: No traceback has been produced, nothing to debug.
